<a href="https://colab.research.google.com/github/nagamamo/financial_data_science/blob/main/7_2_bond_duration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

7.2　債券投資理論とデュレーション

　デュレーションの算出は、償還期間n、クーポンレートci、複利最終利回りr、額面価格Nを定義式に与えることで算出可能です。一方で、即時グロス決済に要する営業日数、年間営業日の設定など、実務の現場で応用するには細かい設定が必要となります。このため本例では数理ファイナンス実務のために開発されたオープンソース・ソフトウェア・ライブラリのQuantLibを用いて実装を行います。まず、このQuantLibソフトウェアをGoogle Colab上で、ひとつのコードセルのみを用いてインストールします。

In [ ]:
pip install QuantLib

　QuantLibがインストールされれば、次にこのライブラリを数値演算ライブラリのnumpyとともにインポートします。

In [2]:
#[1]ライブラリの読み込み
import QuantLib as ql#数理ファイナンス・ライブラリのインポート
import numpy as np

　次に、クーポンレートc、複利最終利回りr、額面価格Nの3つのデータを与えます。本例は確定利付債のため、cは期間を通じてすべて等しい支払（受取）額となります。本例では令和6年度利付国債（第82回債）を参考に、30年債の3条件の設定を下記の通りとしました。

In [4]:
#[2]償還期間・クーポーン・利回りの設定
c = 0.018#クーポン・レート
r = 0.0216#複利最終利回り
N = 100#額面価格

　続いて発行日と償還期間および債券発行スケジュールの詳細を設定します。本例では令和6年に発行された30年国債(第82回債）を用いています。各発行債券ごとに、発行日、償還日を入力してください。国・地域により祝日、非営業日が異なるため、米国債や英国債のコンベクシティを算出する場合は、4行目を
https://quantlib-python-docs.readthedocs.io/en/latest/instruments/bonds.html
に基づき変更すること


In [6]:
#[3]発行日・償還期間・スケジュールの設定
schedule = ql.Schedule(
    ql.Date(16, 6, 2024),       #発行日
    ql.Date(20, 3, 2054),       #償還日
    ql.Period(0, ql.Months),    #間隔
    ql.Japan(),                 #祝休日等の国別カレンダー指定
    ql.Unadjusted,              #非営業日調整
    ql.Unadjusted,              #終了日の非営業日調整
    ql.DateGeneration.Backward, #日付生成のルール
    False,                      #開始日が月末の場合に終了日以外の利払日を月末とするか否か
)

　最後に確定利付債のための関数であるql.FixedRateBond()を用いてデュレーションを算出します。ql.FixedRateBondの（）内には上から順に、決済日数、額面価格、利払い日、クーポン・レート、年間営業日数計算手段、を入力します。本例の決済手段はこれを入力し、債券決済期間として2営業日（T+1）を入力しています。

In [11]:
#[4]デュレーションの算出：債券の条件設定
bond = ql.FixedRateBond(
    2,                  #決済に要する日数
    N,                  #額面
    schedule,           #利払日
    [c],                #クーポンレート
    ql.Actual365Fixed() #日数計算方法
)

　利回り（rate）の設定は、ql.InterestRate()を用い、最終利回り、年間営業日数、複利か否か、年率か否か、を順に指示します。年間営業日数は複数の算出方法も指定可能ですが、本例では以下の手段を用いています。

In [12]:
#[5]デュレーションの算出：利回りの条件設定
rate = ql.InterestRate(
    r,                  #最終利回り
    ql.Actual360(),     #営業日数計算方法
    ql.Compounded,      #複利か否か
    ql.Annual           #年間利回りか否か
)

最後にql.BondFunctions()を用いて,

1.   デュレーション
2.   修正デュレーション
3.   マコーレー・デュレーション

の3種類のデュレーションを算出します。

In [15]:
#[6]デュレーションの算出：実装
norm_duration = ql.BondFunctions.duration(bond, rate, ql.Duration.Simple)#デュレーションの算出
mod_duration = ql.BondFunctions.duration(bond, rate, ql.Duration.Modified)#修正デュレーションの算出
mac_duration = ql.BondFunctions.duration(bond, rate, ql.Duration.Macaulay)#マコーレー・デュレーションの算出

　本例の30年債の修正デュレーションとマコーレー・デュレーションの算出結果は下記の通りです。上記のsource codeに、例えば、令和4年度個人向け国債第147回債（償還期間3年）の3条件を[3]で与えると、デュレーションはより小さくなります。